In [1]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
from datasets import load_dataset, Dataset

dataset = load_dataset("cambridgeltl/xcopa", 'translation-id')

data = {}
data['val'] = dataset['validation'][:]
data['test'] = dataset['test'][:]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [3]:
print(data['val']['premise'][:5])
print(data['val']['question'][:5])

['The man has turned on the tap.', 'The girl had found an insect in her bowl filled with cereal.', 'The woman has retired.', 'I want to save energy.', 'The hamburger meat was browned.']
['effect', 'effect', 'effect', 'effect', 'cause']


In [4]:
# modifikasi agar tasknya menjadi commonsense reasoning generation
# original validation
data['val']['new_premise'] = []
data['val']['new_label'] = []

for i in range(len(data['val']['premise'])):
  cause_or_effect = 'sebab' if data['val']['question'][i] == 'cause' else 'akibat' # translate to indonesian
  data['val']['new_premise'].append(f"Berikan {cause_or_effect} dari kalimat berikut: {data['val']['premise'][i]}")
  data['val']['new_label'].append(data['val']['choice1'][i] if data['val']['label'][i] == 0 else data['val']['choice2'][i])


# original test
data['test']['new_premise'] = []
data['test']['new_label'] = []

for i in range(len(data['test']['premise'])):
  cause_or_effect = 'sebab' if data['test']['question'][i] == 'cause' else 'akibat' # translate to indonesian
  data['test']['new_premise'].append(f"Berikan {cause_or_effect} dari kalimat berikut: {data['test']['premise'][i]}")
  data['test']['new_label'].append(data['test']['choice1'][i] if data['test']['label'][i] == 0 else data['test']['choice2'][i])

In [5]:
print(data['val']['new_premise'][4:6])
print(data['val']['new_label'][4:6])

['Berikan sebab dari kalimat berikut: The hamburger meat was browned.', "Berikan akibat dari kalimat berikut: I doubt the merchant's offer."]
['The chef has baked it.', 'I turned down his offer.']


In [6]:
# ubah menjadi split dataset sesuai yang direncanakan (400:100:100)
data["new_train"] = {"input":data["test"]["new_premise"][:400], "label":data["test"]["new_label"][:400]}
data["new_val"] = {"input":data["test"]["new_premise"][400:], "label":data["test"]["new_label"][400:]}
data["new_test"] = {"input":data["val"]["new_premise"], "label":data["val"]["new_label"]}

data["new_train"] = Dataset.from_dict(data["new_train"])
data["new_val"] = Dataset.from_dict(data["new_val"])
data["new_test"] = Dataset.from_dict(data["new_test"])

In [7]:
print(data["new_train"]['input'][:5])
print(data["new_train"]['label'][:5])

print(len(data["new_train"]))
print(len(data["new_val"]))
print(len(data["new_test"]))

['Berikan sebab dari kalimat berikut: The item is packed in bubble wrap.', 'Berikan akibat dari kalimat berikut: I empty my pockets.', 'Berikan akibat dari kalimat berikut: My house was invaded by termites.', 'Berikan akibat dari kalimat berikut: The tourists have reached the border.', 'Berikan sebab dari kalimat berikut: The office is closed.']
['Stuff is fragile.', 'I took a ticket stub.', 'The termites eat the wood of my house.', 'Their passports are checked by a patrol agent.', 'It was a holiday at that time.']
400
100
100


In [8]:
# Save file for reproducibility (will be publicly visible at https://huggingface.co/datasets/dehanalkautsar/xcopa_gen_id/)

import pandas as pd

df_train = data['new_train'].to_pandas()
df_val = data['new_val'].to_pandas()
df_test = data['new_test'].to_pandas()

df_train.to_csv('train.csv', index=False)
df_val.to_csv('validation.csv', index=False)
df_test.to_csv('test.csv', index=False)

print("Dataset saved")

Dataset saved
